In [ ]:
# !pip install -U langchain-upstage
# !pip install -U langchain-community faiss-cpu

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_upstage import ChatUpstage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_upstage import ChatUpstage
import os

In [ ]:
os.environ["UPSTAGE_API_KEY"] = "up_8cyoE4JaMkvgfFjGncc4tajhqUK1H"

In [ ]:
import glob
files = [file for file in glob.glob("/content/pdf_datasets/*")]
def read_dataset(files):
    all_docs = []
    for file_name in files:
        # Load the file
        file1_load = UpstageLayoutAnalysisLoader(file_name, split="page")
        docs = file1_load.load()
        for doc in docs:
            all_docs.append(doc)
    return all_docs

In [ ]:
# #Load the file
# # file1_load = UpstageLayoutAnalysisLoader(file1, split="page")
# # docs = file1_load.load()
# for doc in docs:
#   print(doc)

In [ ]:
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = UpstageEmbeddings(
  upstage_api_key="up_8cyoE4JaMkvgfFjGncc4tajhqUK1H",
  model="solar-embedding-1-large"
)
# Embed and store all documents in FAISS
docs = read_dataset(files)
db = FAISS.from_documents(docs, embeddings)

In [ ]:
# Asking the retriever to do similarity search based on Query
# query = "Tell me about gender bias"
# answer = db.similarity_search(query)

# Building the retriever
retriever = db.as_retriever(search_kwargs = {'k': 3} )

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'UpstageEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7a105535d690>, search_kwargs={'k': 3})

In [ ]:

template = """

  You are a pdf file information retrieval AI chat assistant. Format the retrieved information as text.


  Use only the context for your answers, do not make up information.

  query: {query}

  {context}
"""
# using chain
prompt = ChatPromptTemplate.from_template(template)
model = ChatUpstage()
chain = (
    {
        "context" : retriever,
        "query" : RunnablePassthrough()
    }
    | prompt | model | StrOutputParser()

)


In [ ]:
chain.invoke(""" What are some of the linguistic factors that helps identifying gender bias? Use all the embedded documents for context.
""")

'To identify gender bias, linguistic factors such as gender-specific pronouns, gendered nouns, and stereotypical associations with certain professions or roles can be used. These factors can be analyzed using Natural Language Processing (NLP) techniques such as Term Frequency-Inverse Document Frequency (TF-IDF) and Principal Component Analysis (PCA). Additionally, the use of gender-neutral language and avoiding stereotypes can help reduce gender bias in language.'

References:
1. https://python.langchain.com/v0.1/docs/integrations/document_loaders/upstage/
2. https://medium.com/firebird-technologies/chat-with-your-pdfs-using-langchain-e57866b7926d